# Task 1: Monte Carlo Simulation
1. Fetch the data from Yahoo Finance
2. Preprocess the date, and calculate return over 5 years
3. Peforme the Monte Carlo simulation
4. Conclude on best strategy

In [ ]:
# Load Neccessary Libraries
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import numpy as np

## 1. Fetch Data

In [ ]:
tickers = ['KO', 'GME', 'AAPL', 'JNJ', 'JPM']

end = datetime(2024, 3, 1) # End day of the analysis
start = end - timedelta(days=365*5 +2) # 5 years and 2 days (leap year 2024 and 2020)
end = int(end.timestamp()) # Convert to unix timestamp
start = int(start.timestamp()) # Convert to unix timestamp

# Load Data from Yahoo Finance
for ticker in tickers:
    url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={start}&period2={end}&interval=1d&events=history&includeAdjustedClose=true'
    df = pd.read_csv(url)
    df['Date'] = pd.to_datetime(df['Date'])
    df['daily_return'] = df['Close'].pct_change() # Calculate daily return
    df.to_csv(f'data/{ticker}.csv', index=False) # Save to csv file

## 2. Preprocess the data
We will start to merge all the .csv files

In [ ]:
df_all = pd.DataFrame() # Create an empty dataframe
for ticker in tickers:
    df = pd.read_csv(f'data/{ticker}.csv') # Read in the csv file
    df['Date'] = pd.to_datetime(df['Date'])
    # Append the df to df_all
    df['ticker'] = ticker # Append a column for ticker name
    df_all = df_all.append(df) # Append to df_all dataframe

# Save the combined dataframe to a csv file
df_all.to_csv('data/stock_data_all.csv', index=False)

To ensure data quaility, we will make a plot to se that we got stock prices for all the assets in the 5 year period

In [ ]:
# Plot the data
df_all.to_csv('data/stock_data_all.csv', index=False)
df_all['Date'] = pd.to_datetime(df_all['Date'])
fig = px.line(df_all, x=df_all['Date'], y='Close', color='ticker', title='Stock Prices')
fig.show()

fig = px.line(df_all, 
    x=df_all['Date'],
    y='daily_return',
    color='ticker', 
    title='Daily Returns (%)')
fig.show()

From the two plots above, we can see that we have data for all the periods.

The next thing we are going to do is to calculate the 5 year return and variance for each assets. This will further be used to calculate the expected return and -variance for the portefolio.

In [ ]:
def calulate_5_year_return_and_variance(ticker):
    """
    Calculate the 5 year return and variance of a stock

    Args:
    ticker (str): The stock ticker
    """
    df = pd.read_csv(f'data/{ticker}.csv')
    five_year_return = (1+ df['daily_return']).prod() -1
    five_year_variance = df['daily_return'].var()
    return five_year_return, five_year_variance

stock_return = []

for ticker in tickers:
    p = []
    p.append(ticker)
    p.append(calulate_5_year_return_and_variance(ticker)[0])
    p.append(calulate_5_year_return_and_variance(ticker)[1])
    stock_return.append(p)
    print(f'{ticker} 5 year return: {round(calulate_5_year_return_and_variance(ticker)[0], 2) * 100} % and variance: {round(calulate_5_year_return_and_variance(ticker)[1], 7)}')

# Save the return and variance to a csv file
stock_return_df = pd.DataFrame(stock_return, columns=['Ticker', '5 Year Return', '5 Year Variance'])
stock_return_df.to_csv('data/stock_return.csv', index=False)


3. Monte carlo simulation

In [ ]:
num_portfolios = 10_000 # The number of portfolios to simulate
return_stock = pd.read_csv('data/stock_return.csv') # Read in the data
tickers = return_stock['Ticker']
risk_free_rate = 0.025  # The risk free rate

# Construct a DataFrame to store the results
results = pd.DataFrame()

# Loop through many portfolios
for i in range(num_portfolios):
    # Randomly assign weights to the assets
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)

    # Calculate expected portfolio return
    expected_return = np.dot(weights, return_stock['5 Year Return'])

    # Calculate portfolio variance
    variance = np.dot(weights.T, np.dot( np.diag(return_stock['5 Year Variance']), weights))

    # Calculate portfolio standard deviation (volatility)
    volatility = np.sqrt(variance)

    # Calculate Sharpe Ratio
    sharpe_ratio = (expected_return - risk_free_rate) / volatility

    # Append results
    portfolio = pd.DataFrame({
        'sim_n': i,
        'KO': weights[0],
        'GME': weights[1],
        'AAPL': weights[2],
        'JNJ': weights[3],
        'JPM': weights[4],
        'returns': expected_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe_ratio
    }, index=[0])
    results = results.append(portfolio)

# Saving the results
results.to_csv('data/portfolios.csv', index=False)


Plotting the distribution over returns

In [ ]:
fig = px.scatter(
    results, 
    x='volatility',
    y='returns',
    color='sharpe_ratio', title='Relative Risk and Return')
fig.show()

fig = px.histogram(
    results, 
    x='returns', 
    title='Distribution of Returns',
    nbins=40)

fig.update_layout(
    xaxis_title='Return (1+%) - 5 Year Return',
    yaxis_title='Frequency')


In [ ]:
portfolio = results[results['sharpe_ratio'] == results['sharpe_ratio'].max()]
print(portfolio)

Based on the above results, we should invest in the follownig portfolio

|Ticker|Weight|
|------|------|
|KO | 15.2%|
|GME| 4.3%
|AAPL|59.2%
|JNJ|10.6%|
|JPM|10.6%|

And this will gives us an expected return of 220%

**NB** Based on randomness in choose of weighet per assets, the portfolio will have minor changes based on every rerun of the model